In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import mlflow
from mlflow.tracking import MlflowClient


In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow
MLFLOW_TRACKING_URI="sqlite:///mlflow.db"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [4]:

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


In [ ]:
client.create_experiment(name="green-taxi-duration")

In [15]:
mlflow.set_experiment("green-taxi-duration")
# mlflow.set_experiment("mlops-zoomcamp-experiment-tracking")

<Experiment: artifact_location='file:///c:/Users/LENOVO/Documents/mlops-zoomcamp/mlops-zoomcamp/04-Deployment/web-service-mlflow/mlruns/1', creation_time=1722296940049, experiment_id='1', last_update_time=1722296940049, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [6]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [7]:
df_train = read_dataframe(r'C:\Users\LENOVO\Documents\mlops-zoomcamp\mlops-zoomcamp\01-intro\data\yellow_tripdata_2023-01.parquet')
df_val = read_dataframe(r'C:\Users\LENOVO\Documents\mlops-zoomcamp\mlops-zoomcamp\01-intro\data\yellow_tripdata_2023-02.parquet')

df_train.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [9]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

MemoryError: Unable to allocate 45.9 MiB for an array with shape (6018346,) and data type float64

In [ ]:
from mlflow.tracking import MlflowClient


In [16]:
# MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '7b805331916a444c8dcf0c63497eab85'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [17]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

OSError: No such file or directory: 'C:\Users\LENOVO\Documents\mlops-zoomcamp\mlops-zoomcamp\04-Deployment\web-service-mlflow\mlruns\1\7b805331916a444c8dcf0c63497eab85\artifacts\dict_vectorizer.bin'

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv